In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
# 下载数据集的路径
path = './DataSet'
# 使用的设备
device = torch.device("cuda")
# 数据预处理,转换为张量并归一化
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 将图像大小调整为 224x224
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 归一化处理

train_set = CIFAR10(root=path, train=True, download=True, transform=transform)
test_set = CIFAR10(root=path, train=False, download=True, transform=transform)
# 加载数据集并设置批次大小
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False)
# 打印数据集的信息
print(
    f"训练集的 batch 数量: {len(train_loader)}，每个 batch 中的数据数量: {train_loader.batch_size}")
print(
    f"测试集的 batch 数量: {len(test_loader)}，每个 batch 中的数据数量: {test_loader.batch_size}")

Files already downloaded and verified
Files already downloaded and verified
训练集的 batch 数量: 196，每个 batch 中的数据数量: 256
测试集的 batch 数量: 40，每个 batch 中的数据数量: 256


In [ ]:
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Flatten
from torch.nn import Sequential
from torch.nn import AdaptiveAvgPool2d
import torch

from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_


class GoogLeNet(Module):
    class Inception(Module):
        # c1--c4是每条路径的输出通道数
        def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
            super(GoogLeNet.Inception, self).__init__(**kwargs)
            # 线路1，单1x1卷积层
            self.p1 = Sequential(
                Conv2d(in_channels, c1, kernel_size=1),
                ReLU()
            )
            kaiming_uniform_(self.p1[0].weight, nonlinearity='relu')
            # 线路2，1x1卷积层后接3x3卷积层
            self.p2 = Sequential(
                Conv2d(in_channels, c2[0], kernel_size=1),
                ReLU(),
                Conv2d(c2[0], c2[1], kernel_size=3, padding=1),
                ReLU()
            )
            kaiming_uniform_(self.p2[0].weight, nonlinearity='relu')
            kaiming_uniform_(self.p2[2].weight, nonlinearity='relu')
            # 线路3，1x1卷积层后接5x5卷积层
            self.p3 = Sequential(
                Conv2d(in_channels, c3[0], kernel_size=1),
                ReLU(),
                Conv2d(c3[0], c3[1], kernel_size=5, padding=2),
                ReLU()
            )
            kaiming_uniform_(self.p3[0].weight, nonlinearity='relu')
            kaiming_uniform_(self.p3[2].weight, nonlinearity='relu')
            # 线路4，3x3最大汇聚层后接1x1卷积层
            self.p4 = Sequential(
                MaxPool2d(kernel_size=3, stride=1, padding=1),
                Conv2d(in_channels, c4, kernel_size=1),
                ReLU()
            )
            kaiming_uniform_(self.p4[1].weight, nonlinearity='relu')

        def forward(self, x):
            p1 = self.p1(x)
            p2 = self.p2(x)
            p3 = self.p3(x)
            p4 = self.p4(x)
            # 在通道维度上连结输出
            return torch.cat((p1, p2, p3, p4), dim=1)

    def __init__(self):
        super().__init__()
        self.block1 = Sequential(
            Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        kaiming_uniform_(self.block1[0].weight, nonlinearity='relu')
        self.block2 = Sequential(
            Conv2d(64, 64, kernel_size=1),
            ReLU(),
            Conv2d(64, 192, kernel_size=3, padding=1),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        kaiming_uniform_(self.block2[0].weight, nonlinearity='relu')
        kaiming_uniform_(self.block2[2].weight, nonlinearity='relu')
        self.block3 = Sequential(
            GoogLeNet.Inception(192, 64, (96, 128), (16, 32), 32),
            GoogLeNet.Inception(256, 128, (128, 192), (32, 96), 64),
            MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.block4 = Sequential(
            GoogLeNet.Inception(480, 192, (96, 208), (16, 48), 64),
            GoogLeNet.Inception(512, 160, (112, 224), (24, 64), 64),
            GoogLeNet.Inception(512, 128, (128, 256), (24, 64), 64),
            GoogLeNet.Inception(512, 112, (144, 288), (32, 64), 64),
            GoogLeNet.Inception(528, 256, (160, 320), (32, 128), 128),
            MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.block5 = Sequential(
            GoogLeNet.Inception(832, 256, (160, 320), (32, 128), 128),
            GoogLeNet.Inception(832, 384, (192, 384), (48, 128), 128),
            AdaptiveAvgPool2d((1, 1)),
            Flatten(),
            Linear(1024, 10)
        )
        kaiming_uniform_(self.block5[4].weight, nonlinearity='relu')

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        return x

In [ ]:
# 训练模型
# 定义损失函数和优化器
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import time
from torch.amp import GradScaler, autocast


def train_model(train_loader, model):
    # 定义损失函数和优化器
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # 训练的轮数
    num_epochs = 10
    # 初始化 running_loss
    running_loss = 0.0
    # 记录训练开始时间
    start_time = time.time()

    # 混合精度训练的梯度缩放器
    scaler = GradScaler()
    # 梯度累积步数
    accumulation_steps = 1

    # 遍历 epoch
    for epoch in range(num_epochs):
        # 遍历训练数据
        for i, (inputs, labels) in enumerate(train_loader):
            # 将数据移动到 GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            with autocast('cuda'):
                # 计算模型输出
                y_hat = model(inputs)
                # 计算损失
                loss = criterion(y_hat, labels)
                # 梯度累积
                loss = loss / accumulation_steps

            # 反向传播
            scaler.scale(loss).backward()

            # 梯度累积更新
            if (i + 1) % accumulation_steps == 0:
                # 更新参数
                scaler.step(optimizer)
                scaler.update()
                # 梯度清零
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps

            # 释放中间变量和清空缓存
            del inputs, labels, y_hat, loss
            torch.cuda.empty_cache()

        # 打印每个 epoch 的损失
        print(
            f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')
        # 每个 epoch 结束后重置 running_loss
        running_loss = 0.0
    # 记录训练结束时间
    end_time = time.time()
    # 计算训练总时间
    total_time = end_time - start_time
    # 转换为分钟和秒
    minutes = int(total_time // 60)
    seconds = total_time % 60
    print(
        f'Training completed in {minutes} minutes and {seconds:.2f} seconds.')

In [4]:
# 评估模型
from sklearn.metrics import accuracy_score
from numpy import vstack
from numpy import argmax


def evaluate_model(test_loader, model):
    predictions, actuals = list(), list()
    for i, (inputs, labels) in enumerate(test_loader):
        # 将数据移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 计算模型输出
        y_hat = model(inputs)
        # 转换为 numpy 数据类型
        y_hat = y_hat.detach().cpu().numpy()
        actual = labels.cpu().numpy()
        # 转换为类标签
        y_hat = argmax(y_hat, axis=1)
        # 为stack格式化
        actual = actual.reshape((len(actual), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        # 存储
        predictions.append(y_hat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # 计算准确率
    acc = accuracy_score(actuals, predictions)
    return acc

In [9]:
# 产生实例,并且将实例放入GPU
GoogLeNet_model = GoogLeNet().to(device)

# 训练实例
train_model(train_loader, GoogLeNet_model)
# 评估实例整体准确率
acc = evaluate_model(test_loader, GoogLeNet_model)
print('Overall Accuracy: %.3f' % acc)

Epoch 1/10, Loss: 1.92929972921099
Epoch 2/10, Loss: 1.5499276293783772
Epoch 3/10, Loss: 1.3434604345535746
Epoch 4/10, Loss: 1.2010833140538664
Epoch 5/10, Loss: 1.0634814902227752
Epoch 6/10, Loss: 0.9486277039561953
Epoch 7/10, Loss: 0.8491951874932464
Epoch 8/10, Loss: 0.776390801887123
Epoch 9/10, Loss: 0.6944063094501592
Epoch 10/10, Loss: 0.6386481030863158
Training completed in 12 minutes and 20.51 seconds.
Overall Accuracy: 0.690
